In [2]:
import kagglehub
import pandas as pd

In [1]:
%pip install kaggle

In [3]:
from google.colab import files
files.upload()  # Upload kaggle.json when prompted

# Move kaggle.json to the .kaggle directory
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [4]:
!kaggle datasets download -d aryansingh0909/nyt-articles-21m-2000-present

Dataset URL: https://www.kaggle.com/datasets/aryansingh0909/nyt-articles-21m-2000-present
License(s): CC0-1.0
100% 872M/873M [00:15<00:00, 92.9MB/s]
100% 873M/873M [00:15<00:00, 60.4MB/s]


In [6]:
import zipfile
path = "/content/NYT DATA"
with zipfile.ZipFile('nyt-articles-21m-2000-present.zip', 'r') as zip_ref:
    zip_ref.extractall(path)  # Adjust the path if necessary

In [8]:
import os
os.listdir(path)

['nyt-metadata.csv', 'negative-words.txt', 'positive-words.txt']

In [10]:
df = pd.read_csv(path+"/nyt-metadata.csv")

<ipython-input-10-3be1698fda21>:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path+"/nyt-metadata.csv")


In [11]:
articles_only = df[df['document_type'] == 'article']
articles_only = articles_only.dropna(subset=['section_name'])
print(articles_only['section_name'].unique())

['Sports' 'Opinion' 'U.S.' 'Archives' 'New York' 'Business Day'
 'Technology' 'Arts' 'Books' 'World' 'Theater' 'Education' 'Movies'
 'Travel' 'Style' 'Real Estate' 'Magazine' 'Week in Review' 'Automobiles'
 'Fashion & Style' 'Health' 'Science' 'Job Market' 'Food' 'Home & Garden'
 'T Magazine' 'Giving' 'Obituaries' 'The Learning Network' 'membercenter'
 'Climate' 'Your Money' 'Corrections' 'Homepage' 'readersopinions'
 'Crosswords & Games' 'Great Homes & Destinations' 'Multimedia/Photos'
 'Washington' 'College' 'none' 'Blogs' 'Times Topics' 'Well' 'UrbanEye'
 "Critic's Choice" 'xword' 'Sunday Review' 'Public Editor' 'Booming'
 'International Home' 'The Upshot' 'Times Insider' 'NYT Now' 'Universal'
 'Topics' 'Today’s Paper' 'Podcasts' 'Briefing' 'Watching' 'Admin'
 'Smarter Living' 'Reader Center' 'Lens' 'Video' 'Neediest Cases'
 'Parenting' 'The Weekly' 'Guides' 'At Home' 'en Español' 'Special Series'
 'The New York Times Presents' 'Headway' 'Book Review' 'Weather']


I think we should get rid of Obituaries because there is no really good/bad sentiment. Member center because it just seems like maintenance stuff. Archives because they are from a bunch of different print sections so for our purpose it does not give value (cant easily separate topics).Corrections because it is just correcting a previous article. May give duplicated points for articles. Crosswords & games because while some are good articles.. too many are about the actual crosswords and games in the NYT and have nonsense abstracts.'xword' is crossword answers that was only used 3 times. I want to get rid of 'Today's Paper but am having issues. Reader Center is the NYT responding to reader comments basically. I think thiis could double sentiments for the same article again. Homepage is short and useless. 'readersopinions' is summaries from reader forums-- again I think it would repeat the content of some articles. 'Admin' seems to be a mix between admin stuff for corrections and cooking stuff?? I think we'll have to review this one more maybe. For now, removing. 'Week in Review' will 1. have repeated sentiments from articles that week, 2. Will have a mix of sentiments from all the articles that week. 'Critic's Choice' is basically like little riddles-- OUT.


In [42]:
curr_sect = 'Podcasts'
removeList = ['Obituaries', 'membercenter','Archives','Corrections','Crosswords & Games','none','xword','Reader Center','Homepage','readersopinions','Admin','Week in Review','Critic\'s Choice','Topics']

rowWithSelection = articles_only[articles_only['section_name'] == curr_sect]
rowWithSelection.head(5)
# print(rowWithSelection.head(1)['abstract'].values[0])
print(rowWithSelection.iloc[20]['abstract'])
# print(rowWithSelection.iloc[5]['web_url'])

David Barstow and Susanne Craig explain how they cracked open the biggest story of the campaign, overcoming nagging doubts to nail down Donald Trump’s tax returns.


In [35]:
from transformers import pipeline


In [18]:
val = 8
# Load a sentiment-analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

# Text to analyze
text = df['snippet'][val]
print(text)

# Run sentiment analysis
result = sentiment_pipeline(text)[0]

print(result)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Judge D Irvin Couvillion rules that Burton W Kanter, one of nation's most prominent tax lawyers, was 'architect' of long-running kickback and tax-evasion scheme in which money was siphoned from funds managed by Prudential and Travelers insurance c...
{'label': 'NEGATIVE', 'score': 0.9134017825126648}


In [43]:
#cleaning
print("Starting Shape: ",df.shape)
df = df.dropna(subset = ['abstract'])
print("Shape after dropping NaN for abstract:",df.shape) #One of these must have a value to do a sentiment analysis I think we prioritize abstract then snippet then lead
df = df.dropna(subset = ['keywords'])
print("Shape after dropping for keywords:",df.shape) #need key words to do keyword analysis
df = df.dropna(subset = ['section_name'])  #no awesome mapping for print section to section name. Only keeping section name
print("Shape after dropping for print section or section name:",df.shape) #If we want to compare sections, then at least one of these must have a value
df = df.dropna(subset = ['web_url','headline'])
print("Shape after dropping for web_url and headline:",df.shape) #want to title and link articles if user wants to read the articles from certain keywords
df = df = df[df['document_type'] == 'article']
print("Shape after only choosing articles:",df.shape) #we only want articles right? Not audio or multimedia?
df = df.dropna(subset = ['pub_date'])
print("Shape after dropping for pub date:", df.shape) #need a date to plot on a timeline
df = df.dropna(subset=['_id'])
print("Shape after dropping for id:",df.shape)
df = df[~((df['section_name'] == 'The Learning Network') & (df['abstract'].str.contains('.menu', na=False)))]
print("Shape after cleaning Learning Network",df.shape)
df = df[~df['section_name'].isin(removeList)]
print("Shape after removing removeList from column (section_name):",df.shape)


Starting Shape:  (1904974, 20)
Shape after dropping NaN for abstract: (1904974, 20)
Shape after dropping for keywords: (1904974, 20)
Shape after dropping for print section or section name: (1904974, 20)
Shape after dropping for web_url and headline: (1904974, 20)
Shape after only choosing articles: (1904974, 20)
Shape after dropping for pub date: (1904974, 20)
Shape after dropping for id: (1904974, 20)
Shape after cleaning Learning Network (1904974, 20)
Shape after removing removeList from column (section_name): (1904973, 20)


In [54]:
articles = df #work with smaller set to make sure the analysis works before we use for
sentiment_pipeline = pipeline("sentiment-analysis") #create sentiment pipeline

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
#Negative : -1 , POSITIVE: 1
#create new column in df with pos/neg and another with score
articles['sentiment'] = 0
articles['sentiment_score'] = 0.0
error_indices = []

for index,row in articles.iterrows():
  try:
    sentiment = sentiment_pipeline(row['abstract'])[0]
    articles.at[index,'sentiment'] = 1 if sentiment['label'] == 'POSITIVE' else -1
    articles.at[index,'sentiment_score'] = sentiment['score']
  except Exception as e:
    print(f"An unexpected error occurred: {e}")
    error_indices.extend(articles.index[articles['_id'].str.contains(row['_id'])].tolist())

print("Error Indices:\n",error_indices)

Token indices sequence length is longer than the specified maximum sequence length for this model (1043 > 512). Running this sequence through the model will result in indexing errors


An unexpected error occurred: The size of tensor a (1043) must match the size of tensor b (512) at non-singleton dimension 1
An unexpected error occurred: The size of tensor a (1406) must match the size of tensor b (512) at non-singleton dimension 1
An unexpected error occurred: The size of tensor a (1149) must match the size of tensor b (512) at non-singleton dimension 1
An unexpected error occurred: The size of tensor a (2216) must match the size of tensor b (512) at non-singleton dimension 1
